This document describes the commercial parking structure model and runs a number of examples to illustrate its use and key features.

# Commercial Parking Structures
Commercial parking structures can be generally described as multi-level buildings that provides a large number of parking spots, a subset of which are equipped with electric vehicle chargers.  Parking structures may be equipped with lights and fans.  Multi-level parking structures may also have elevators to transport people between levels.

Total power demand depends on lighting, fans, elevators and electric vehicle chargers.

Revenue depends on fixed and hourly parking fees, and electricity supply for vehicle connected to chargers.

## Parking Structure Parameters

- $Total Size$: (static input) the total number of parking spots installed
- $Total Chargers$: (static input) the total number of electric vehicle chargers installed
- $Total Level$: (static input) the total number of levels in the parking structure
- $Total Elevators$: (static input) the total number of elevators in the parking structure

## Vehicle Parameters
- $Total Cars$: (variant input) the total number of cars present in the parking structure
- $Ev Count$: (variant input) the total number of EVs connected in the parking
- $Ev Active$: (variant input) the number of connected EVs actively charging
- $Ev Fraction$: (variant input) the fraction of vehicles that are electric

## Power Parameters
- $ElevatorUsage$: (static input) the fractional increase in elevator usage per level
- $ElevatorUnitPower$: (variant input) the power required to move an elevator
- $ElevatorUnitEnergy$: (variant input) the energy required to move an elevator per level
- $ChargerUnitPower$: (variant input) the power draw of one charger when active
- $LightingPower$: (output) the total lighting power draw
- $ElevatorPower$: (output) the total elevator power draw
- $FanPower$: (output) the total ventilation fan power draw
- $ChargerPower$: (output) the total vehicle charger power draw

## Cost Parameters
- $Parking Fee$: (variant input) the hourly parking fee
- $Charger Price$: (variant input) the energy charging fee
- $Fee Waiver$: (variant input) the hourly parking fee waiver for charger occupancy
- $Idle Penalty$: (variant input) the hourly parking fee for idle charger occupancy
- $Total Revenue$: (output) the total revenue

## Usage Parameters
- $Elevator Usage$: the probability of using an elevator per level
- $Parking log$: (output) the log of parking transactions
- $Arrivals$: (variant input) the vehicle arrival rate
- $Departures$: (variant input) the vehicle departure rate

## Rules of Association
The load composition is obtained from the end-uses as follows:

| Component  | Lighting | Fans | Elevators | Chargers |
| :--------- | :------: | :--: | :-------: | :------: |
| Motor A    |    -     |  -   |     -     |    -     |
| Motor B    |    -     | 1.0  |     -     |    -     |
| Motor C    |    -     |  -   |    1.0    |    -     |
| Motor D    |    -     |  -   |     -     |    -     |
| Static I   |   1.0    |  -   |     -     |    -     |
| Static R   |    -     |  -   |     -     |    -     |
| Electronic |    -     |  -   |     -     |   1.0    |

# Load Modeling
The load model is updated when one of the following events occurs.

## 1. Vehicle Arrival
Vehicle arrival is a random event based on the vehicle arrival rate $Arrivals$. 

When elevators are present, a vehicle arrival may result in an elevator load.  The model assumes that cars fill the parking from the ground floor. Thus when the parking lot is empty, there is little or no elevator use, and when the parking lot is full, there is significant elevator use. For each arrival the following logic is observed:

- The level on which the car parks is a function of the fractional usage of the parking structure, i.e., $Level = \mathrm{E}(TotalCars / TotalSize)$.

- The probability of a non-zero elevator load is $\rho = Level \times ElevatorUsage$ or $1.0$, whichever is less.

- If $\rho > Uniform(0,1)$, then the elevator energy is $Level \times ElevatorUnitEnergy$

- The elevator run time is $t = ElevatorEnergy/ElevatorPower$.

- TODO: handle more than one elevator by saturating $ElevatorPower$ based on $TotalElevators$.

- If $EvFraction < Uniform(0,1)$ and $EvActive < TotalChargers$, then $EvActive$ is incremented and $ChargerPower = EvActive \times ChargerUnitPower$.

## 2. Vehicle Departure
Vehicle departure is a random event based on the vehicle departure rate $Departures$.

When elevators are present, a vehicle departure may result in an elevator load. The model assumes that cars depart the parking uniformly from all floors.

- The probability that a car will depart from a level is inversely proportional to the number of a levels, e.g., $Level = \mathrm{E}(Uniform(0,ParkingSize))$.

- The probability of a non-zero elevator load is $\rho = Level \times ElevatorUsage$ or $1.0$, whichever is less.

- If $\rho > Uniform(0,1)$, then the elevator energy is $Level \times ElevatorUnitEnergy$

- The elevator run time is $t = ElevatorEnergy/ElevatorPower$.

- TODO: handle more than one elevator by saturating $ElevatorPower$ based on $TotalElevators$.

- If $EvFraction < Uniform(0,1)$ and $EvActive > 0$, then $EvActive$ is decrements and $ChargerPower = EvActive \times ChargerUnitPower$.

## 3. Vehicle Charged

TODO: address vehicle full-charge events